In [1]:
import os

from collections import OrderedDict

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import scipy

import warnings

from Standard_functions import plot_features, get_user_split_data, plot_fit_score_pred

warnings.filterwarnings('always') 
np.seterr(divide='ignore', invalid='ignore')
%matplotlib inline

In [2]:
os.chdir('../Data/')

In [3]:
os.listdir()

['mini_feature_df',
 'mini_df',
 'products.csv',
 'orders.csv',
 '.DS_Store',
 '00_merged_prod_order',
 'scaled_poly_feats_without_dep_df.pkl',
 'mini_ord_prod_train_df',
 'poly_feats_without_dep_df.pkl',
 'order_products__train.csv',
 'departments.csv',
 '190208 Decorators.ipynb',
 'full_features_with_dep',
 'aisles.csv',
 'full_features',
 '.ipynb_checkpoints',
 'mini_ord_df',
 'order_products__prior.csv',
 'no_feature_reduction',
 'sample_submission.csv']

## Loading our Datasets

In [21]:
ord_prod_test_df = pd.read_csv('order_products__train.csv')
ord_df = pd.read_csv('orders.csv')
ord_prod_test_df = ord_prod_test_df.merge(ord_df, on='order_id')
prod_df = pd.read_csv('products.csv')
dep_df = pd.read_csv('departments.csv')

In [23]:
ord_prod_test_df.user_id.nunique()

131209

In [13]:
ord_prod_test_df.eval_set.unique()

array(['train'], dtype=object)

In [14]:
test = pd.read_csv('order_products__prior.csv')

In [26]:
test.sort_values('order_id')

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
16,3,32665,8,1


In [5]:
df = (ord_prod_test_df.groupby(['product_id','user_id'],as_index=False) 
                              .agg({'order_id':'count'}) 
                              .rename(columns={'order_id':'num_ord_per_user_per_prod'}))

df.head()

,product_id,user_id,num_ord_per_user_per_prod
0,1,751,1
1,1,1540,1
2,1,2221,1
3,1,9694,1
4,1,13916,1


In [20]:
df.user_id.nunique()

131209

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8474661 entries, 0 to 8474660
Data columns (total 5 columns):
product_id                   int64
user_id                      int64
num_ord_per_user_per_prod    int64
latest_cart                  object
in_cart                      int64
dtypes: int64(4), object(1)
memory usage: 387.9+ MB


## Feature Engineering

Our feature engineering began in the 'Organizing data section.  
We will visualize that first feature below and begin creating new features.

As you can see, we're setting the bar pretty low.

## Product Features
Note that product features are not user-specific.

### `product_total_orders`
Total number of orders placed for a given product.
### `product_avg_add_to_cart_order`
Average order in which a product was add to a users cart.

In [10]:
prod_features = ['product_total_orders','product_avg_add_to_cart_order']

df_prod_features = (ord_prod_train_df.groupby(['product_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'),
                                                    ('add_to_cart_order','mean')])))
df_prod_features.columns = ['product_id'] + prod_features
df = df.merge(df_prod_features, on='product_id')

`product_total_orders` had some pretty enourmous rightward skew that ultimately resulted in the model producing 0 positive predictions. As a result I took the log base 2 transformation to punish outliers.  
Note that I attempted a log base 10 transformation but log base 2 had better results.

In [11]:
df.product_total_orders = np.log2(df.product_total_orders)

## User Features
`'user_total_orders','user_avg_cartsize','user_total_products','user_avg_days_since_prior_order'`

In [13]:
user_features = ['user_total_orders','user_avg_cartsize',
                 'user_total_products','user_avg_days_since_prior_order']

df_user_features = (ord_prod_train_df.groupby(['user_id'],as_index=False)
                    .agg(OrderedDict(
                       [('order_id',
                         ['nunique', # user_total_orders
                        (lambda x: x.shape[0] / x.nunique())]), # user_avg_cartsize
                        ('product_id','nunique'), # user_total_products
                        ('days_since_prior_order','mean')]))) # user_avg_days_since_prior_order
df_user_features.columns = ['user_id'] + user_features
df = df.merge(df_user_features, on='user_id')

## User-Product features

In [15]:
user_prod_features = ['user_product_avg_add_to_cart_order']

df_user_prod_features = (ord_prod_train_df.groupby(
                        ['product_id','user_id'],as_index=False) \
                        .agg(OrderedDict(
                        [('add_to_cart_order','mean')])))

df_user_prod_features.columns = ['product_id','user_id'] + user_prod_features 
df = df.merge(df_user_prod_features,on=['user_id','product_id'])
df['user_product_order_freq'] = df['num_ord_per_user_per_prod'] / df['user_total_orders'] 

In [17]:
#pd.to_pickle(df,'mini_feature_df')

## Number of Orders Placed Since a Product was Last Ordered
## Percentage of Orders Containing a Product

In [18]:
new_feats = ['total_orders_containig_product','most_recent_order_by_user_by_product']

most_recent_order_by_user_by_product = (ord_prod_train_df.groupby(['user_id','product_id'],
                                                        as_index=False).agg(OrderedDict(
                                                        [('reordered','sum'),
                                                        ('order_number','max')])))
most_recent_order_by_user_by_product.columns = ['user_id','product_id'] + new_feats

max_orders_per_user = ord_prod_train_df.groupby(['user_id'],as_index=False).order_number.max()
max_orders_per_user.columns = ['user_id'] + ['number_orders_since_product_last_ordered']

number_orders_since_product_last_ordered = pd.merge(most_recent_order_by_user_by_product, 
                                                    max_orders_per_user,on='user_id')

number_orders_since_product_last_ordered['percent_of_time_product_ordered'] =(
    number_orders_since_product_last_ordered['total_orders_containig_product'] / 
    number_orders_since_product_last_ordered['number_orders_since_product_last_ordered'])
    # Note that we use the column name `number_orders_since_product_last_ordered` as that
    # is currently set to the max max order and does not yet actually contain the 
    # number of orders since a product was last ordered.

percent_of_time_product_ordered = number_orders_since_product_last_ordered[['percent_of_time_product_ordered',
                                                                      'user_id','product_id']]

number_orders_since_product_last_ordered = (number_orders_since_product_last_ordered[
                    ['most_recent_order_by_user_by_product','number_orders_since_product_last_ordered']]
                    .diff(axis=1).drop('most_recent_order_by_user_by_product',axis=1))

most_recent_order_by_user_by_product.drop('most_recent_order_by_user_by_product',
                                          axis=1,inplace=True) # we're just using this to get back our product_id and user_id
number_orders_since_product_last_ordered = pd.merge(number_orders_since_product_last_ordered,most_recent_order_by_user_by_product,
                        left_index=True,right_index=True)

combined_feats = pd.merge(number_orders_since_product_last_ordered,percent_of_time_product_ordered,
                        on=['user_id','product_id'])

combined_feats.drop('total_orders_containig_product',axis=1,inplace=True)

df = pd.merge(df, combined_feats,on=['user_id','product_id'])

## Time since most recent order by user

In [20]:
mean_hod = (ord_df.groupby(['user_id'],as_index=False)
                    .agg({'order_hour_of_day':'mean'}))
mean_hod.columns = ['user_id','mean_order_hour_of_day_by_user']

most_recent_order_time = (ord_df.groupby(['user_id'],as_index=False)
                    .agg({'order_number':'max'}))
temp = pd.merge(most_recent_order_time,ord_df, on=['user_id','order_number'],how='left')
temp = pd.merge(temp[['user_id','order_hour_of_day']], mean_hod, on=['user_id'])
temp['diff_between_average_and_current_order_time'] = (temp['order_hour_of_day'] - 
                                        temp['mean_order_hour_of_day_by_user'])
df = pd.merge(df, temp.drop('order_hour_of_day',axis=1)
              ,on=['user_id'])

## Weighting products for more of more recent orders

In [22]:
recent_order_weight = ord_prod_train_df.loc[:,['product_id', 'reordered', 'user_id', 'order_number']]
max_order_number = ord_df.groupby(['user_id'],as_index=False).agg({'order_number':'max'})
max_order_number.columns = ['user_id','max_order_number']
recent_order_weight = recent_order_weight.merge(max_order_number,on='user_id')
# Here we set max_order_number to be the nth traingular number of the max order number
recent_order_weight['max_order_number'] = ((recent_order_weight.max_order_number *
                                            (recent_order_weight.max_order_number + 1)) / 2)
recent_order_weight['recent_order_weight'] = ((recent_order_weight.reordered 
                                              * recent_order_weight.order_number)
                                              / recent_order_weight.max_order_number)
recent_order_weight = recent_order_weight.groupby(['product_id','user_id'],as_index=False).sum()
recent_order_weight.drop(['reordered','order_number'],axis=1,inplace=True)
df = pd.merge(df, recent_order_weight,on=['user_id','product_id'])

### Removing Max Order Number
Max order number clearly had a significant negative impact and has been removed.

In [ ]:
df.drop(['max_order_number'],axis=1,inplace=True)

In [28]:
pd.to_pickle(df,'full_features_test')

## Adding Department Dummy Variable

In [24]:
department_name_product_id = (prod_df.merge(dep_df,on='department_id')
                              .drop(['product_name','aisle_id','department_id'],axis=1))
dep_dummies = pd.get_dummies(department_name_product_id,drop_first=False)
dummies_cols = dep_dummies.columns.tolist()
new_cols = []
for name in dummies_cols:
    new_cols.append(str.replace(name,' ','_'))
dep_dummies.columns = new_cols

dep_dummies['positvely_corr_departments'] = dep_dummies.department_dairy_eggs + dep_dummies.department_produce
dep_dummies['negatively_corr_departments'] = dep_dummies.department_pantry + dep_dummies.department_personal_care
dep_dummies = dep_dummies[['negatively_corr_departments','positvely_corr_departments','product_id']]

df = pd.merge(df, dep_dummies,on=['product_id'])

In [28]:
pd.to_pickle(df,'full_features_with_dep_test')